In [2]:
import sys

sys.path.append("..")

In [3]:
import functools
import os
import pickle
from typing import Any, Tuple

import numpy as np
import torch
import torch.nn.functional as F
from model import DiffWave
from omegaconf import OmegaConf
from rfcutils2 import qpsk_helper_fn as qpskfn
from utils import view_as_complex


/fs/data/tejasj/anaconda3/envs/rf_tf_torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-05 20:04:01.091534: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-05 20:04:01.656545: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-02-05 20:04:01.656599: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnv

In [4]:
device = torch.device("cuda:0")

In [5]:
# Load the DiffWave QPSK model
checkpoint = torch.load("../checkpoints/updated/qpsk_100000_160_20_03_09/weights-360000.pt")
cfg_qpsk = OmegaConf.create(checkpoint["cfg"])
model_qpsk = DiffWave(cfg_qpsk.model).to(device)
model_qpsk.load_state_dict(checkpoint["model"])
model_qpsk.eval()

print()

In [6]:
# Create a colored Gaussian covariance matrix
cov_b = pickle.load(open('ofdm_cov.pkl', 'rb'))
# Compute the Cholesky decomposition
L = np.linalg.cholesky(cov_b)

In [7]:
# Create a new colored Gaussian signal
z = (np.random.randn(L.shape[1], 1) + 1j * np.random.randn(L.shape[1], 1)) / np.sqrt(2)
w = L @ z
w = w.flatten()

In [8]:
# Load the interference root directory
interference_type = "qpsk"
qpsk_root_dir = "../dataset/separation/qpsk/"
b = np.load(os.path.join(qpsk_root_dir, "sig_200.npy"))

In [35]:
cov_b_tensor = torch.view_as_real(torch.tensor(cov_b, device=device)).float()
inv_cov_b_tensor = torch.view_as_real(torch.linalg.inv(torch.tensor(cov_b, device=device))).float()

In [9]:
def matchedfilter_remod(sig):
    b, _ = qpskfn.qpsk_matched_filter_demod(sig)
    s, _, _, _ = qpskfn.modulate_qpsk_signal(b)
    return s

In [28]:
def complex_mm(x, y):
    # x: (d, d, 2)
    # y: (b, 2, d)
    real_component = torch.einsum("ijl,blj->bi", x, y * torch.tensor([[1], [-1]], device=device))
    imag_component = torch.einsum("ijl,blj->bi", x, torch.flip(y, dims=[1]))
    return torch.stack([real_component, imag_component], dim=1)

In [42]:
MIN_STEP, MAX_STEP = 1, 49


def args_separation(
        mixture: torch.Tensor, 
        coeff: float, 
        scaling: float, 
        num_iters: int, 
        use_trained_model: bool, 
        soi: torch.Tensor, 
        interference: torch.Tensor,
        model_interference: torch.nn.Module,
        learning_rate_range: Tuple[float, float],
        dataset_stats: Tuple[torch.Tensor, torch.Tensor],
        device: Any,
        **kwargs,
):
    """Perform source separation using \\alpha-RGS.

    Args:
        mixture: Inputs mixture (y = s + \kappa * b)
        coeff: \kappa
        scaling: \\alpha-posterior hyperparameter (\omega)
        num_iters: Number of iterations to run \\alpha-RGS (N)
        use_trained_model: If True, use the learned diffusion model to compute
            the SOI score
        soi: The SOI (s)
        interference: The interference (b)
        model_interference: The interference diffusion model
        learning_rate_range: Learning rate schedule range (\eta_max, \eta_min)
        dataset_stats: Dataset mean and standard deviation if interference is 
            not zero mean and unit variance.
    """

    del kwargs
    eta_max, eta_min = learning_rate_range
    # Same noise schedule as in our diffusion models. See learner.py
    noise_schedule = np.linspace(1e-4, 0.05, 50)
    noise_level = torch.tensor(
        np.cumprod(1 - noise_schedule).astype(np.float32)
    ).to(device)

    # Compute the matched filtering solution given the mixture
    b_mf = matchedfilter_remod(view_as_complex(coeff * mixture))
    b_est = torch.view_as_real(torch.tensor(b_mf.numpy())).transpose(1, 2).float().to(device)
    mixture = mixture.to(device)
    
    for i in range(num_iters):
        # Update learning rate using cosine annealing learning rate schedule
        eta = eta_min + 0.5 * (eta_max -  eta_min)*(1 + np.cos(np.pi * (i / num_iters)))

        # Sample u and 1 - \alpha_u
        t2 = torch.randint(MIN_STEP, max(1, int(MAX_STEP)) + 1, [1], device=device)
        noise_scale2 = noise_level[t2]
        
        noise2 = torch.randn_like(b_est)
        
        # Apply Gaussian smoothing to the estimate for b
        b_est = (b_est - dataset_stats[0]) / dataset_stats[1]
        bu_est = noise_scale2 ** 0.5 * b_est.to(device) + (1 - noise_scale2) ** 0.5 * noise2.to(device)
        # Apply Gaussian smoothing to the estimates for s
        s_est = (mixture * coeff - b_est).to(device)
                
        # Compute the score of the smoothened interference
        sigma2 = (1 - noise_scale2.detach()) ** 0.5 
        score_t2 = model_qpsk(bu_est, t2)
        score_t2 = score_t2 - noise2.to(device)
        score_t2 = -score_t2.detach()/(sigma2)
        score_t2 *= (1 - sigma2 ** 2) ** 0.5
        
        # Compute the score of the SOI
        score_t1 = -complex_mm(inv_cov_b_tensor, s_est)
        
        g = -scaling * score_t2 + score_t1

        # Update the estimate using stochastic gradient descent (SGD)
        b_est = b_est - eta * g

        if (i+1) % 1000  == 0:
            b_hat = b_est
            s_hat = (mixture * coeff - b_hat)
            bit_true, _ = qpskfn.qpsk_matched_filter_demod(view_as_complex(interference))
            bit_pred, _ = qpskfn.qpsk_matched_filter_demod(view_as_complex(b_hat.detach().cpu()))
            bit_error = np.mean(bit_true != bit_pred)
            print(
                f"{i:>4}: - MSE(s_hat-s): {F.mse_loss(soi, s_hat.detach().cpu()):.4f},"
                f" MSE(b_hat-b): {F.mse_loss(interference, b_hat.detach().cpu()):.6f},"
                f" BER: {bit_error:.4f}"
            )
    return s_est

In [43]:
sir_db = -31
scaling = "kappa"
use_trained_model = False

In [44]:
# Load the inference dataset and pre-trained diffusion models
if interference_type == "commsignal2":
    dataset_mean = torch.tensor(
        np.array([ 1.3608e-05, -1.5107e-06]).reshape(1, 2, 1)
    ).float()
    dataset_std =  torch.tensor(
        np.array([0.7634, 0.7634]).reshape(1, 2, 1)
    ).float()
else:
    dataset_mean = torch.zeros(1, 2, 1)
    dataset_std = torch.ones(1, 2, 1)

# Set the value of \kappa
coeff = np.sqrt(10 ** (-sir_db / 10))
if "ofdm" in interference_type:
    # Only used for plotting, not required for source separation
    coeff = coeff * np.sqrt(64/56)  # 56/64 is the compensating power for OFDM

# Set the value of \alpha-posterior parameter \alpha=\omega
scaling_str = scaling
use_alpha_posterior = False
if isinstance(scaling, str):
    if scaling == "invkappa":
        scaling = 1 / coeff
    elif scaling == "kappa":
        scaling = coeff
        use_alpha_posterior = True
    else:
        raise ValueError("Unexpected string identifier for scaling value.")

separation_fn = functools.partial(
    args_separation,
    coeff=coeff,
    scaling=scaling,
    num_iters=10000,
    use_trained_model=use_trained_model,
    model_interference=model_qpsk,
    cfg_interference=cfg_qpsk,
    learning_rate_range=[5e-3, 1e-6],
    dataset_stats=(dataset_mean.to(device), dataset_std.to(device)),
    use_alpha_posterior=use_alpha_posterior,
    device=device,
)

# Create an output file to save results 
output_folder = "metrics/QPSK_ColoredGaussian_aRGS"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
print(f"Experiments for {output_folder} at {sir_db} dB SINR")

interference = torch.view_as_real(torch.tensor(b)).transpose(0, 1).float().unsqueeze(0)
soi = torch.view_as_real(torch.tensor(w)).transpose(0, 1).float().unsqueeze(0)

# Normalize the mixture by \kappa, i.e., y / \kappa
y = 1 / coeff * soi + interference
with torch.no_grad():
    # SOI estimate using source separator
    s_pred = separation_fn(
        mixture=y, 
        soi=soi, 
        interference=interference,
    )
    s_pred = s_pred.to(device)

# Dump all the results into the output folder
pickle.dump(
    (y, soi, interference, s_pred), 
    open(f"{output_folder}/sample_{sir_db}dB_{idx}.pkl","wb")
)

bit_true, _ = qpskfn.qpsk_matched_filter_demod(view_as_complex(interference.cpu()))
bit_pred, _ = qpskfn.qpsk_matched_filter_demod(view_as_complex(s_pred.cpu()))

print("=================================================")
print(f"SIR [dB] = {sir_db}, sample number {idx - 100}")
print(f"BER (aRSG) = {np.mean(bit_true != bit_pred)}")
print(f"MSE (aRSG) = {F.mse_loss(soi, s_pred.detach().cpu()).numpy()}")
print("=================================================")
print()

Experiments for metrics/QPSK_ColoredGaussian_aRGS at -31 dB SINR


  0%|          | 0/100 [00:00<?, ?it/s]

 999: - MSE(s_hat-s): 591.2035, MSE(b_hat-b): 1.456908, BER: 0.2219
1999: - MSE(s_hat-s): 588.2611, MSE(b_hat-b): 1.404494, BER: 0.2719
2999: - MSE(s_hat-s): 585.3636, MSE(b_hat-b): 0.875354, BER: 0.1594
3999: - MSE(s_hat-s): 705.5793, MSE(b_hat-b): 23.423038, BER: 0.6813
4999: - MSE(s_hat-s): 682.5434, MSE(b_hat-b): 20.907093, BER: 0.5938
5999: - MSE(s_hat-s): 595.1469, MSE(b_hat-b): 0.826368, BER: 0.2594
6999: - MSE(s_hat-s): 608.6193, MSE(b_hat-b): 0.630489, BER: 0.3000
7999: - MSE(s_hat-s): 619.6227, MSE(b_hat-b): 0.700731, BER: 0.3375
8999: - MSE(s_hat-s): 620.2085, MSE(b_hat-b): 0.751516, BER: 0.4094


  1%|          | 1/100 [01:13<2:00:59, 73.33s/it]

9999: - MSE(s_hat-s): 619.4652, MSE(b_hat-b): 0.745540, BER: 0.4094
SIR [dB] = -31, sample number 0
BER (aRSG) = 0.0
MSE (aRSG) = 619.4678955078125

 999: - MSE(s_hat-s): 574.5735, MSE(b_hat-b): 3.744415, BER: 0.2062
1999: - MSE(s_hat-s): 604.7513, MSE(b_hat-b): 0.717318, BER: 0.2938
2999: - MSE(s_hat-s): 616.5256, MSE(b_hat-b): 0.759187, BER: 0.3469
3999: - MSE(s_hat-s): 598.0924, MSE(b_hat-b): 0.679680, BER: 0.2406
4999: - MSE(s_hat-s): 618.9128, MSE(b_hat-b): 0.512804, BER: 0.3406
5999: - MSE(s_hat-s): 597.4986, MSE(b_hat-b): 0.784416, BER: 0.2750
6999: - MSE(s_hat-s): 654.2134, MSE(b_hat-b): 1.765303, BER: 0.7969
7999: - MSE(s_hat-s): 618.3211, MSE(b_hat-b): 0.586669, BER: 0.3156
8999: - MSE(s_hat-s): 619.1688, MSE(b_hat-b): 0.735480, BER: 0.3906


...

9999: - MSE(s_hat-s): 623.1461, MSE(b_hat-b): 0.834936, BER: 0.4375
SIR [dB] = -31, sample number 9
BER (aRSG) = 0.0
MSE (aRSG) = 623.1488647460938

 999: - MSE(s_hat-s): 583.5298, MSE(b_hat-b): 1.385865, BER: 0.2281
1999: - MSE(s_hat-s): 781.1377, MSE(b_hat-b): 84.935387, BER: 0.6813
2999: - MSE(s_hat-s): 651.9216, MSE(b_hat-b): 30.929987, BER: 0.4500
3999: - MSE(s_hat-s): 616.1300, MSE(b_hat-b): 0.665578, BER: 0.2969
4999: - MSE(s_hat-s): 588.8362, MSE(b_hat-b): 1.074302, BER: 0.2406
5999: - MSE(s_hat-s): 601.9276, MSE(b_hat-b): 0.873642, BER: 0.2281
6999: - MSE(s_hat-s): 620.8665, MSE(b_hat-b): 0.884471, BER: 0.5437
7999: - MSE(s_hat-s): 634.5305, MSE(b_hat-b): 1.095688, BER: 0.6188
8999: - MSE(s_hat-s): 615.8720, MSE(b_hat-b): 0.556390, BER: 0.2687


 11%|█         | 11/100 [13:29<1:49:23, 73.75s/it]

9999: - MSE(s_hat-s): 611.8992, MSE(b_hat-b): 0.531845, BER: 0.2687
SIR [dB] = -31, sample number 10
BER (aRSG) = 0.0
MSE (aRSG) = 611.9017944335938

 999: - MSE(s_hat-s): 697.3035, MSE(b_hat-b): 45.002872, BER: 0.6219


 11%|█         | 11/100 [13:41<1:50:44, 74.66s/it]


KeyboardInterrupt: 